In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import sys
import os
import os.path as osp
import cv2
from vehicle_reid.embedder import Embedder
from vehicle_reid.evaluate import Evaluator
from vehicle_reid import exp_config
from vehicle_reid.common import *
from sift_match.sift_match import *
from matplotlib import pyplot as plt
from classifier import classifier_common as clscom
import tensorflow as tf
import numpy as np
import h5py
import sklearn

In [2]:
MODE = "query"
VIEW = "vehicle_view_prediction/040519_{}_view_0.96.csv".format(MODE)
CLS_TYPE = "vehicle_type_prediction/030519_{}_type_0.92.csv".format(MODE)
FRCNN = "frcnn_prediction/frcnn_{}.csv".format(MODE)
GT = "GT/GT_vehi_type_{}.csv".format(MODE)
img_id, img_type_predict, img_type_score = clscom.read_csv_file(CLS_TYPE)

In [3]:
img_view_score = []
with open(VIEW) as fi:
    csv_reader = csv.reader(fi, delimiter = ',')
    for info in csv_reader:
        img_view_score.append(info[1].split(' '))
img_view_score = np.array(img_view_score).astype(np.float32)

frcnn_img_type_predict = []
frcnn_img_type_score = []
frcnn_mapper = {
    'car' : 0,
    'truck': 3,
    'bus': 2,
    '-1': -1
}
with open(FRCNN) as fi:
    csv_reader = csv.reader(fi,delimiter = ',')
    for info in csv_reader:
        frcnn_img_type_predict.append(frcnn_mapper[info[1]])
        frcnn_img_type_score.append(float(info[2]))
img_gt = []
with open(GT) as fi:
    for info in fi:
        img_gt.append(int(info.strip()))
img_gt = np.array(img_gt)
frcnn_img_type_score = np.array(frcnn_img_type_score)
frcnn_img_type_predict = np.array(frcnn_img_type_predict)

In [4]:
is_pick_up = img_gt == 1
is_truck = img_gt == 3
is_truck_or_pick_up = np.logical_or(is_pick_up, is_truck)

In [110]:
frcnn_img_type_score > 0.4

array([ True,  True,  True, ...,  True,  True,  True])

In [39]:
#Bounding box with label "truck" and conf > thresh
conclusion_truck_or_pickup = np.logical_and(frcnn_img_type_predict == 3, frcnn_img_type_score > 0.0)
#View conf
_view_conf = np.logical_and(img_view_score[:,2] > 0.9, img_view_score[:,1] > 0.0) #view2: side, view 1: rear
conclusion_truck_or_pickup = np.logical_and(conclusion_truck_or_pickup, _view_conf)
print("PREDICT-COUNT ", np.sum(conclusion_truck_or_pickup))
_correct = np.logical_and(conclusion_truck_or_pickup, is_truck_or_pick_up)
_incorrect = np.logical_xor(conclusion_truck_or_pickup, is_truck_or_pick_up)
print("CORRECT {}/{}".format(np.sum(_correct), np.sum(is_truck_or_pick_up)))
print("IN-CORRECT ",np.sum(conclusion_truck_or_pickup) - np.sum(_correct))

PREDICT-COUNT  131
CORRECT 68/152
IN-CORRECT  63


In [41]:
_view_conf[0]

False

In [47]:
with open("frcnn_total.csv", "w") as fo:
    for i in range(len(img_id)):
        fo.write("{},{},{},{},{},{},{}\n".format(
            img_id[i], 
            img_gt[i],
            frcnn_img_type_predict[i],
            frcnn_img_type_score[i],
            img_view_score[i][1],
            img_view_score[i][2],
            img_view_score[i][0]
        ))